# Overview
This script aims to pridict list of mitre attack technique given a techinque to detect. This will help to give the use a means to know which techniques run following a script that was choses. Eg. User picks T1027 and runs the sript. if the technique is detected in the log, then he is recommended with a list of other techniques to run a search on. 

The algoritm follows a statistical approach where it finds the most number of times the technique were used togather. The dataset used for this statistical analysis is done by using APT actors from MITRE Attack repository. Once we get the common pairs, we form a weighted graph and then using nearest neighbour finder to find the list of techniques that is related to each other.

In [2]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pprint import pprint
from collections import Counter
from pyattck import Attck
import networkx as nx

In [3]:
# Init pyatt lib
attck = Attck()

In [4]:
# Gather and store edge pairs
sourcelist = []
targetlist = []

for actor in attck.enterprise.actors:
    edge_i = [x.id for x in actor.techniques]
    edge_j = [x.id for x in actor.techniques]
    
    for technique_i in edge_i:
        for technique_j in edge_j:
            if technique_j != technique_i:
                sourcelist.append(technique_i)
                targetlist.append(technique_j)

In [7]:
# User counter to count number of time edges appear to create weights
counter = Counter(tuple(sorted(tup)) for tup in zip(sourcelist,targetlist))
counter

Counter({('T1027', 'T1566.001'): 76,
         ('T1053.005', 'T1566.001'): 50,
         ('T1204.002', 'T1566.001'): 110,
         ('T1566.001', 'T1571'): 14,
         ('T1036.004', 'T1566.001'): 18,
         ('T1059.005', 'T1566.001'): 60,
         ('T1105', 'T1566.001'): 76,
         ('T1027', 'T1053.005'): 56,
         ('T1027', 'T1204.002'): 78,
         ('T1027', 'T1571'): 18,
         ('T1027', 'T1036.004'): 22,
         ('T1027', 'T1059.005'): 54,
         ('T1027', 'T1105'): 86,
         ('T1053.005', 'T1204.002'): 48,
         ('T1053.005', 'T1571'): 12,
         ('T1036.004', 'T1053.005'): 18,
         ('T1053.005', 'T1059.005'): 36,
         ('T1053.005', 'T1105'): 54,
         ('T1204.002', 'T1571'): 14,
         ('T1036.004', 'T1204.002'): 20,
         ('T1059.005', 'T1204.002'): 60,
         ('T1105', 'T1204.002'): 76,
         ('T1036.004', 'T1571'): 8,
         ('T1059.005', 'T1571'): 16,
         ('T1105', 'T1571'): 18,
         ('T1036.004', 'T1059.005'): 12,
         (

In [68]:
# Store each value in array list
sourcelist = []
targetlist = []
weightlist = []
for key, value in counter.items():
    sourcelist.append(key[0])
    targetlist.append(key[1])
    weightlist.append(-abs(value))

In [69]:
weightlist

[-76,
 -50,
 -110,
 -14,
 -18,
 -60,
 -76,
 -56,
 -78,
 -18,
 -22,
 -54,
 -86,
 -48,
 -12,
 -18,
 -36,
 -54,
 -14,
 -20,
 -60,
 -76,
 -8,
 -16,
 -18,
 -12,
 -18,
 -58,
 -28,
 -28,
 -28,
 -40,
 -18,
 -4,
 -26,
 -22,
 -14,
 -12,
 -10,
 -10,
 -10,
 -26,
 -10,
 -12,
 -2,
 -26,
 -34,
 -6,
 -2,
 -18,
 -16,
 -36,
 -10,
 -4,
 -22,
 -12,
 -10,
 -12,
 -4,
 -12,
 -6,
 -18,
 -6,
 -6,
 -2,
 -20,
 -28,
 -2,
 -2,
 -24,
 -44,
 -28,
 -2,
 -20,
 -18,
 -12,
 -18,
 -8,
 -8,
 -10,
 -26,
 -12,
 -16,
 -6,
 -18,
 -32,
 -4,
 -4,
 -36,
 -22,
 -6,
 -24,
 -22,
 -12,
 -10,
 -6,
 -12,
 -12,
 -24,
 -12,
 -10,
 -6,
 -18,
 -22,
 -4,
 -4,
 -42,
 -6,
 -42,
 -34,
 -20,
 -26,
 -10,
 -18,
 -16,
 -48,
 -14,
 -18,
 -6,
 -36,
 -64,
 -4,
 -4,
 -6,
 -24,
 -24,
 -14,
 -18,
 -6,
 -8,
 -14,
 -34,
 -12,
 -12,
 -6,
 -22,
 -32,
 -4,
 -4,
 -4,
 -4,
 -4,
 -4,
 -4,
 -4,
 -4,
 -6,
 -4,
 -2,
 -2,
 -4,
 -2,
 -4,
 -2,
 -22,
 -12,
 -14,
 -6,
 -14,
 -10,
 -26,
 -8,
 -16,
 -6,
 -20,
 -36,
 -4,
 -2,
 -20,
 -16,
 -8,
 -12,
 -16,
 -34,
 -6,
 -10,

In [70]:
# check if the len are same value
print(len(sourcelist))
print(len(targetlist))
print(len(weightlist))

19201
19201
19201


In [71]:
# Store the nodes and weights in dataframe
edges = pd.DataFrame(
    {
        "source": sourcelist,#sourceId,
        "target": targetlist, #targetlist
        "weight": weightlist
    }
)
edges

,source,target,weight
0,T1027,T1566.001,-76
1,T1053.005,T1566.001,-50
2,T1204.002,T1566.001,-110
3,T1566.001,T1571,-14
4,T1036.004,T1566.001,-18
...,...,...,...
19196,T1003.003,T1568.001,-2
19197,T1106,T1583.001,-2
19198,T1106,T1568.001,-2
19199,T1083,T1568.001,-2


# Nearest neighbour based

In [92]:
# Create a graph using the dataframe !! DOES NOT WORK
G = nx.from_pandas_edgelist(edges, edge_attr=True, create_using=nx.DiGraph())
# Run prediction on techniques
technic_neighbours = G.neighbors('T1542.002')
technic_neighbours = [x for x in technic_neighbours]
for item in technic_neighbours:
    for technique in attck.enterprise.techniques:
        if item == technique.id:
            print(item, technique.name, technique.tactics)
        for subtech in technique.subtechniques:
            if item == subtech.id:
                print(item, technique.name, technique.tactics)

T1564.005 Hide Artifacts [AttckTactic('Defense Evasion', 'TA0005')]


# Negative weight shortest path algo

In [81]:
# get all unique combination of path using weight derived from degree of source node
startnode = 'T1566.001'
distance, path = nx.algorithms.single_source_bellman_ford(G, startnode)

In [87]:
# get weights for all shortest path
for x in path.values():
    templist = []
    for i in x:
        templist.append((i, distance[i]))
    print(templist)

[('T1566.001', 0)]
[('T1566.001', 0), ('T1566.002', -52), ('T1566.003', -58), ('T1569.002', -60), ('T1570', -68), ('T1571', -70)]
[('T1566.001', 0), ('T1566.002', -52), ('T1566.003', -58), ('T1569.002', -60), ('T1570', -68), ('T1571', -70), ('T1573.001', -74), ('T1574.002', -80), ('T1583.001', -86)]
[('T1566.001', 0), ('T1566.002', -52), ('T1566.003', -58), ('T1569.002', -60), ('T1570', -68), ('T1571', -70), ('T1573.001', -74), ('T1574.002', -80), ('T1583.001', -86), ('T1583.006', -94), ('T1584.001', -96)]
[('T1566.001', 0), ('T1566.002', -52)]
[('T1566.001', 0), ('T1566.002', -52), ('T1566.003', -58), ('T1569.002', -60), ('T1570', -68), ('T1571', -70), ('T1573.001', -74), ('T1574.002', -80), ('T1583.001', -86), ('T1583.006', -94), ('T1585', -96), ('T1585.001', -98), ('T1585.002', -100)]
[('T1566.001', 0), ('T1566.002', -52), ('T1566.003', -58), ('T1569.002', -60), ('T1570', -68), ('T1571', -70), ('T1573.001', -74), ('T1574.002', -80), ('T1583.001', -86), ('T1583.006', -94), ('T1585', 

In [83]:
# check a path with given end node above
endnode = 'T1571'
for x in path[endnode]:
    print(x, distance[x])

T1566.001 0
T1566.002 -52
T1566.003 -58
T1569.002 -60
T1570 -68
T1571 -70


In [99]:
# get technique names from a spcefic path
paths = list(path.values())[6]
for item in paths:
    for technique in attck.enterprise.techniques:
        if item == technique.id:
            print(item, technique.name, technique.tactics)
        for subtech in technique.subtechniques:
            if item == subtech.id:
                print(item, technique.name, technique.tactics)

T1566.001 Phishing [AttckTactic('Initial Access', 'TA0001')]
T1566.002 Phishing [AttckTactic('Initial Access', 'TA0001')]
T1566.003 Phishing [AttckTactic('Initial Access', 'TA0001')]
T1569.002 System Services [AttckTactic('Execution', 'TA0002')]
T1570 Lateral Tool Transfer [AttckTactic('Lateral Movement', 'TA0008')]
T1571 Non-Standard Port [AttckTactic('Command and Control', 'TA0011')]
T1573.001 Encrypted Channel [AttckTactic('Command and Control', 'TA0011')]
T1574.002 Hijack Execution Flow [AttckTactic('Defense Evasion', 'TA0005'), AttckTactic('Persistence', 'TA0003'), AttckTactic('Privilege Escalation', 'TA0004')]
T1583.001 Acquire Infrastructure [AttckTactic('Resource Development', 'TA0042')]
T1583.006 Acquire Infrastructure [AttckTactic('Resource Development', 'TA0042')]
T1585 Establish Accounts [AttckTactic('Resource Development', 'TA0042')]
T1585.001 Establish Accounts [AttckTactic('Resource Development', 'TA0042')]
T1587.001 Develop Capabilities [AttckTactic('Resource Developmen